In [1]:
# Cài đặt thư viện nếu chưa có
!pip install evaluate rouge_score



  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24972 sha256=b1f562813348bde7c002b26308fc76fc392b5eb2ebba5600e87d6f8553ef9996
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\1e\19\43\8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score

   -------------------- ------------------- 1/2 [evaluate]
   -------------------- ------------------- 1/2 [evaluate]
   -------------------- ------------------- 1/2 [evaluate]
   ---------------------------------------- 2/2 [evaluate]



  DEPRECATION: Building 'rouge_score' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'rouge_score'. Discussion can be found at https://github.com/pypa/pip/issues/6334

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
from datasets import load_dataset
import random

# Load dataset
ds = load_dataset("OpenHust/vietnamese-summarization", split="train")  # hoặc split phù hợp nếu biết rõ

# Lấy ngẫu nhiên 30 dòng
random.seed(42)
sample = ds.shuffle(seed=42).select(range(30))

# Chuyển thành DataFrame
df = pd.DataFrame({
    "original_text": sample["Document"],       # tên trường có thể khác tùy dataset
    "reference_summary": sample["Summary"] # tên trường có thể khác
})

# Lưu thành CSV
df.to_csv("test.csv", index=False, encoding="utf-8-sig")

print("Đã lưu 30 dòng ngẫu nhiên vào file test.csv")


Đã lưu 30 dòng ngẫu nhiên vào file test.csv


In [1]:
import pandas as pd
import evaluate
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
from tqdm import tqdm

# =======================================
# 🧩 1. Đọc dữ liệu test
# =======================================
df = pd.read_csv("test.csv", encoding="utf-8-sig")
texts = df["original_text"].tolist()
references = df["reference_summary"].tolist()

# =======================================
# 🧩 2. Hàm sinh tóm tắt
# =======================================
def generate_summary(model, tokenizer, text, max_new_tokens=100):
    prompt = f"Văn bản: {text}\nTóm tắt:"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048).to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Lấy phần sau "Tóm tắt:"
    if "Tóm tắt:" in result:
        result = result.split("Tóm tắt:")[-1].strip()
    return result

# =======================================
# 🧩 3. Load các mô hình
# =======================================
print("Đang tải mô hình gốc (Qwen/Qwen3-0.6B)...")
model_ori = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-0.6B", device_map="auto")
tokenizer_ori = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B")

print("Đang tải mô hình SFT...")
model_sft_base = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-0.6B", device_map="auto")
model_sft = PeftModel.from_pretrained(model_sft_base, "./runs/qwen3-0.6B-summarization-lora")
tokenizer_sft = AutoTokenizer.from_pretrained("./runs/qwen3-0.6B-summarization-lora")

print("Đang tải mô hình DPO...")
model_dpo_base = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-0.6B", device_map="auto")
model_dpo = PeftModel.from_pretrained(model_dpo_base, "./runs/dpo-qwen3-0.6B")
tokenizer_dpo = AutoTokenizer.from_pretrained("./runs/dpo-qwen3-0.6B")

# =======================================
# 🧩 4. Sinh tóm tắt cho từng mô hình
# =======================================
print("\nSinh tóm tắt từ mô hình gốc...")
pred_ori = [generate_summary(model_ori, tokenizer_ori, t) for t in tqdm(texts)]

print("\nSinh tóm tắt từ mô hình SFT...")
pred_sft = [generate_summary(model_sft, tokenizer_sft, t) for t in tqdm(texts)]

print("\nSinh tóm tắt từ mô hình DPO...")
pred_dpo = [generate_summary(model_dpo, tokenizer_dpo, t) for t in tqdm(texts)]

# =======================================
# 🧩 5. Tính điểm ROUGE
# =======================================
rouge = evaluate.load("rouge")

def calc_rouge(predictions, references):
    scores = rouge.compute(predictions=predictions, references=references)
    return {k: round(v, 4) for k, v in scores.items()}

scores_ori = calc_rouge(pred_ori, references)
scores_sft = calc_rouge(pred_sft, references)
scores_dpo = calc_rouge(pred_dpo, references)

# =======================================
# 🧩 6. Hiển thị kết quả
# =======================================
df_results = pd.DataFrame([
    {"Mô hình": "Qwen gốc", **scores_ori},
    {"Mô hình": "Qwen SFT (LoRA)", **scores_sft},
    {"Mô hình": "Qwen DPO (RL)", **scores_dpo}
])

print("\n=== KẾT QUẢ ĐÁNH GIÁ ROUGE ===")
print(df_results.to_string(index=False))

# =======================================
# 🧩 7. (Tùy chọn) Lưu tóm tắt ra file CSV
# =======================================
out_df = pd.DataFrame({
    "original_text": texts,
    "reference_summary": references,
    "summary_ori": pred_ori,
    "summary_sft": pred_sft,
    "summary_dpo": pred_dpo
})
out_df.to_csv("summarization_results.csv", index=False, encoding="utf-8-sig")
print("\nĐã lưu kết quả chi tiết vào summarization_results.csv")


c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Đang tải mô hình gốc (Qwen/Qwen3-0.6B)...
Đang tải mô hình SFT...
Đang tải mô hình DPO...


Some parameters are on the meta device because they were offloaded to the cpu.
c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\accelerate\utils\modeling.py:1566: UserWarning: Current model requires 256 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(



Sinh tóm tắt từ mô hình gốc...


100%|██████████| 30/30 [13:37<00:00, 27.24s/it]



Sinh tóm tắt từ mô hình SFT...


100%|██████████| 30/30 [14:01<00:00, 28.05s/it]



Sinh tóm tắt từ mô hình DPO...


100%|██████████| 30/30 [11:45<00:00, 23.53s/it]



=== KẾT QUẢ ĐÁNH GIÁ ROUGE ===
        Mô hình  rouge1  rouge2  rougeL  rougeLsum
       Qwen gốc  0.5104  0.1899  0.2981     0.3180
Qwen SFT (LoRA)  0.5301  0.2294  0.3184     0.3189
  Qwen DPO (RL)  0.5387  0.2304  0.3182     0.3169

Đã lưu kết quả chi tiết vào summarization_results.csv
